fgan 1

In [ ]:
# === Imports ===
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix,
                             precision_score, recall_score, f1_score, roc_auc_score)
import joblib
import copy
import matplotlib.pyplot as plt
import seaborn as sns

# === Load and Preprocess Dataset ===
df = pd.read_csv("new_network_train.csv")
X = df.drop(columns=["ProtocolName"]).values
y = df["ProtocolName"].astype(np.int64).values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, "scaler_fgani.pkl")

# === Stratified Split (Train + Test) ===
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42
)

# === Split Training Data into Labeled + Unlabeled ===
X_labeled, X_unlabeled, y_labeled, _ = train_test_split(
    X_train_full, y_train_full, test_size=0.3, stratify=y_train_full, random_state=42
)

# === Tensor Conversion ===
X_labeled_tensor = torch.tensor(X_labeled, dtype=torch.float32)
y_labeled_tensor = torch.tensor(y_labeled, dtype=torch.long)
X_unlabeled_tensor = torch.tensor(X_unlabeled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# === Constants ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = X.shape[1]
num_classes = len(np.unique(y))
latent_dim = 100
NUM_CLIENTS = 5
label_smooth_real = 0.9

# === Partition Data (Labeled only) ===
labeled_dataset = TensorDataset(X_labeled_tensor, y_labeled_tensor)
partition_sizes = [len(labeled_dataset) // NUM_CLIENTS + (1 if i < len(labeled_dataset) % NUM_CLIENTS else 0) for i in range(NUM_CLIENTS)]
client_partitions = random_split(labeled_dataset, partition_sizes)

# === Utility Functions ===
def one_hot(labels, num_classes):
    return torch.eye(num_classes, device=labels.device)[labels]

def get_confident_entropy_pseudo_labels(model, data, conf_thresh=0.9, entropy_thresh=0.5):
    model.eval()
    data = data.to(device)
    with torch.no_grad():
        outputs = model(data)
        probs = F.softmax(outputs, dim=1)
        confidence, preds = probs.max(dim=1)
        entropy = -torch.sum(probs * torch.log(probs + 1e-10), dim=1)
        mask = (confidence >= conf_thresh) & (entropy <= entropy_thresh)
        return data[mask].cpu(), preds[mask].cpu()

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super().__init__()
        self.gamma = gamma
    def forward(self, input, target):
        logpt = F.log_softmax(input, dim=1)
        pt = torch.exp(logpt)
        loss = (1 - pt) ** self.gamma * logpt
        return F.nll_loss(loss, target)

# === Models ===
class Generator(nn.Module):
    def __init__(self, noise_dim, label_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(noise_dim + label_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
        )
    def forward(self, noise, labels):
        labels = one_hot(labels, num_classes).to(noise.device)
        x = torch.cat([noise, labels], dim=1)
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim, label_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim + label_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    def forward(self, data, labels):
        labels = one_hot(labels, num_classes).to(data.device)
        x = torch.cat([data, labels], dim=1)
        return self.model(x)

class CNNClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        return self.net(x)

# === Training Functions ===
def train_fgani_client(client_data, global_G):
    D = Discriminator(input_dim, num_classes).to(device).train()
    loader = DataLoader(client_data, batch_size=64, shuffle=True)
    opt_D = optim.Adam(D.parameters(), lr=0.0002)
    bce = nn.BCELoss()

    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        z = torch.randn(xb.size(0), latent_dim).to(device)
        real_labels = torch.full((xb.size(0), 1), label_smooth_real).to(device)
        fake_labels = torch.zeros((xb.size(0), 1)).to(device)
        fake_data = global_G(z, yb)
        D_real = D(xb, yb)
        D_fake = D(fake_data.detach(), yb)
        loss_D = bce(D_real, real_labels) + bce(D_fake, fake_labels)
        opt_D.zero_grad(); loss_D.backward(); opt_D.step()
    return D

def update_global_generator(global_G, client_discriminators):
    global_G.train()
    opt_G = optim.Adam(global_G.parameters(), lr=0.0002)
    bce = nn.BCELoss()
    for D in client_discriminators:
        D.eval()
        labels = torch.randint(0, num_classes, (64,), device=device)
        z = torch.randn(64, latent_dim, device=device)
        fake_data = global_G(z, labels)
        output = D(fake_data, labels)
        loss = bce(output, torch.full_like(output, label_smooth_real))
        opt_G.zero_grad(); loss.backward(); opt_G.step()

# === Main Training Loop ===
global_G = Generator(latent_dim, num_classes, input_dim).to(device)
global_C = CNNClassifier(input_dim, num_classes).to(device)
scheduler_C = optim.lr_scheduler.StepLR(optim.Adam(global_C.parameters()), step_size=15, gamma=0.5)
focal_loss = FocalLoss()

EPOCHS = 30
for epoch in range(EPOCHS):
    print(f"\n=== Epoch {epoch + 1} ===")
    local_D_models = []

    for client_data in client_partitions:
        D = train_fgani_client(client_data, global_G)
        local_D_models.append(D)

    update_global_generator(global_G, local_D_models)

    # === Classifier Training on Synthetic + Real Data ===
    global_C.train()
    opt_C = optim.Adam(global_C.parameters(), lr=0.001)
    all_data, all_labels = [], []

    for client_data in client_partitions:
        loader = DataLoader(client_data, batch_size=64, shuffle=True)
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            z = torch.randn(xb.size(0), latent_dim).to(device)
            fake_data = global_G(z, yb).detach()
            all_data.append(torch.cat([xb, fake_data], dim=0))
            all_labels.append(torch.cat([yb, yb], dim=0))

    train_loader = DataLoader(TensorDataset(torch.cat(all_data), torch.cat(all_labels)), batch_size=128, shuffle=True)
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        preds = global_C(xb)
        loss = focal_loss(preds, yb)
        opt_C.zero_grad(); loss.backward(); opt_C.step()
    scheduler_C.step()

    # === Pseudo-label Unlabeled Data ===
    pseudo_data, pseudo_labels = get_confident_entropy_pseudo_labels(global_C, X_unlabeled_tensor)
    if len(pseudo_data) > 1:
        print(f"Adding {len(pseudo_data)} pseudo-labeled samples to training.")
        semi_loader = DataLoader(
            TensorDataset(torch.cat([X_labeled_tensor, pseudo_data]), torch.cat([y_labeled_tensor, pseudo_labels])),
            batch_size=128, shuffle=True
        )
        opt_C_pseudo = optim.Adam(global_C.parameters(), lr=0.001)
        for xb, yb in semi_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = global_C(xb)
            loss = focal_loss(preds, yb)
            opt_C_pseudo.zero_grad(); loss.backward(); opt_C_pseudo.step()

# === Final Evaluation ===
global_C.eval()
with torch.no_grad():
    preds = global_C(X_test_tensor.to(device))
    y_pred = preds.argmax(dim=1).cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()

print("\n✅ Accuracy:", accuracy_score(y_true, y_pred))
print("🔍 Precision:", precision_score(y_true, y_pred, average="macro"))
print("🔁 Recall:", recall_score(y_true, y_pred, average="macro"))
print("🎯 F1-Score:", f1_score(y_true, y_pred, average="macro"))
print("\n📊 Classification Report:\n", classification_report(y_true, y_pred))
print("\n🧮 Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

# === Save Models ===
torch.save(global_G.state_dict(), "fgan1_generator.pth")
torch.save(global_C.state_dict(), "fgan1_classifier.pth")



=== Epoch 1 ===


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Adding 3497 pseudo-labeled samples to training.

=== Epoch 2 ===
Adding 2286 pseudo-labeled samples to training.

=== Epoch 3 ===
Adding 5580 pseudo-labeled samples to training.

=== Epoch 4 ===
Adding 6613 pseudo-labeled samples to training.

=== Epoch 5 ===
Adding 6753 pseudo-labeled samples to training.

=== Epoch 6 ===
Adding 7669 pseudo-labeled samples to training.

=== Epoch 7 ===
Adding 7051 pseudo-labeled samples to training.

=== Epoch 8 ===
Adding 8003 pseudo-labeled samples to training.

=== Epoch 9 ===
Adding 8904 pseudo-labeled samples to training.

=== Epoch 10 ===
Adding 8931 pseudo-labeled samples to training.

=== Epoch 11 ===
Adding 9085 pseudo-labeled samples to training.

=== Epoch 12 ===
Adding 8568 pseudo-labeled samples to training.

=== Epoch 13 ===
Adding 8764 pseudo-labeled samples to training.

=== Epoch 14 ===
Adding 8630 pseudo-labeled samples to training.

=== Epoch 15 ===
Adding 7167 pseudo-labeled samples to training.

=== Epoch 16 ===
Adding 8684 pseudo

fgan2

In [ ]:
# === FGAN-II with Feature Matching Loss and Federated Generator/Discriminator ===
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
import joblib
import copy

# === Load and Preprocess Dataset ===
df = pd.read_csv("new_network_train.csv")
X = df.drop(columns=["ProtocolName"]).values
y = df["ProtocolName"].astype(np.int64).values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, "scaler_fgani.pkl")

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# === Constants ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = X.shape[1]
num_classes = len(np.unique(y))
latent_dim = 100
NUM_CLIENTS = 5
label_smooth_real = 0.9

# === Partition Data for Federated Clients ===
dataset = TensorDataset(X_train_tensor, y_train_tensor)
partition_sizes = [len(dataset) // NUM_CLIENTS + (1 if i < len(dataset) % NUM_CLIENTS else 0) for i in range(NUM_CLIENTS)]
client_partitions = random_split(dataset, partition_sizes)

# === Utility Functions ===
def one_hot(labels, num_classes):
    return torch.eye(num_classes, device=labels.device)[labels]

def get_confident_entropy_pseudo_labels(model, data, conf_thresh=0.9, entropy_thresh=0.5):
    model.eval()
    data = data.to(device)
    with torch.no_grad():
        outputs = model(data)
        probs = F.softmax(outputs, dim=1)
        confidence, preds = probs.max(dim=1)
        entropy = -torch.sum(probs * probs.log(), dim=1)
        mask = (confidence >= conf_thresh) & (entropy <= entropy_thresh)
        return data[mask].cpu(), preds[mask].cpu()

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0):
        super().__init__()
        self.gamma = gamma
    def forward(self, input, target):
        logpt = F.log_softmax(input, dim=1)
        pt = torch.exp(logpt)
        loss = (1 - pt) ** self.gamma * logpt
        return F.nll_loss(loss, target)

# === Models ===
class Generator(nn.Module):
    def __init__(self, noise_dim, label_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(noise_dim + label_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
        )
    def forward(self, noise, labels):
        labels = one_hot(labels, num_classes).to(noise.device)
        x = torch.cat([noise, labels], dim=1)
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim, label_dim):
        super().__init__()
        self.feature = nn.Sequential(
            nn.Linear(input_dim + label_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    def forward(self, data, labels):
        labels = one_hot(labels, num_classes).to(data.device)
        x = torch.cat([data, labels], dim=1)
        features = self.feature(x)
        return self.classifier(features), features

class CNNClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        return self.net(x)

# === Federated Averaging ===
def federated_average(models, sizes):
    avg_model = copy.deepcopy(models[0])
    total = sum(sizes)
    for key in avg_model.state_dict():
        weighted_sum = sum(model.state_dict()[key] * (size / total) for model, size in zip(models, sizes))
        avg_model.state_dict()[key].copy_(weighted_sum)
    return avg_model

# === Train Generator + Discriminator on Client ===
def train_fgan2_client(client_data, global_G, global_D):
    G = copy.deepcopy(global_G).train()
    D = copy.deepcopy(global_D).train()
    loader = DataLoader(client_data, batch_size=64, shuffle=True)
    opt_D = optim.Adam(D.parameters(), lr=0.0001)
    opt_G = optim.Adam(G.parameters(), lr=0.0002)
    bce = nn.BCELoss()

    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        batch_size = xb.size(0)
        real_labels = torch.full((batch_size, 1), label_smooth_real).to(device)
        fake_labels = torch.zeros((batch_size, 1)).to(device)

        # Train Discriminator
        z = torch.randn(batch_size, latent_dim).to(device)
        fake_data = G(z, yb)
        D_real_out, _ = D(xb, yb)
        D_fake_out, _ = D(fake_data.detach(), yb)
        loss_D = bce(D_real_out, real_labels) + bce(D_fake_out, fake_labels)
        opt_D.zero_grad(); loss_D.backward(); opt_D.step()

        # Train Generator (Feature Matching Loss)
        _, real_features = D(xb, yb)
        _, fake_features = D(fake_data, yb)
        loss_G = F.mse_loss(fake_features.mean(dim=0), real_features.mean(dim=0))
        opt_G.zero_grad(); loss_G.backward(); opt_G.step()

    return G, D

# === Initialize Models ===
global_G = Generator(latent_dim, num_classes, input_dim).to(device)
global_D = Discriminator(input_dim, num_classes).to(device)
global_C = CNNClassifier(input_dim, num_classes).to(device)
scheduler_C = optim.lr_scheduler.StepLR(optim.Adam(global_C.parameters()), step_size=15, gamma=0.5)
focal_loss = FocalLoss()

# === Training Loop ===
EPOCHS = 30
for epoch in range(EPOCHS):
    print(f"\n=== Epoch {epoch + 1} ===")
    local_G, local_D, sizes = [], [], []

    for client_data in client_partitions:
        G_new, D_new = train_fgan2_client(client_data, global_G, global_D)
        local_G.append(G_new)
        local_D.append(D_new)
        sizes.append(len(client_data))

    global_G = federated_average(local_G, sizes).to(device)
    global_D = federated_average(local_D, sizes).to(device)

    # === Train Classifier ===
    global_C.train()
    opt_C = optim.Adam(global_C.parameters(), lr=0.001)
    all_data, all_labels = [], []
    for client_data in client_partitions:
        loader = DataLoader(client_data, batch_size=64, shuffle=True)
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            z = torch.randn(xb.size(0), latent_dim).to(device)
            fake_data = global_G(z, yb).detach()
            combined_x = torch.cat([xb, fake_data], dim=0)
            combined_y = torch.cat([yb, yb], dim=0)
            all_data.append(combined_x)
            all_labels.append(combined_y)

    train_loader = DataLoader(TensorDataset(torch.cat(all_data), torch.cat(all_labels)), batch_size=128, shuffle=True)
    for xb, yb in train_loader:
        if xb.shape[0] < 2: continue
        xb, yb = xb.to(device), yb.to(device)
        preds = global_C(xb)
        loss = focal_loss(preds, yb)
        opt_C.zero_grad(); loss.backward(); opt_C.step()
    scheduler_C.step()

    # === Pseudo-labeling ===
    pseudo_data, pseudo_labels = get_confident_entropy_pseudo_labels(global_C, X_test_tensor)
    if len(pseudo_data) > 1:
        print(f"Adding {len(pseudo_data)} pseudo-labeled samples to training.")
        semi_loader = DataLoader(
            TensorDataset(torch.cat([X_train_tensor, pseudo_data]), torch.cat([y_train_tensor, pseudo_labels])),
            batch_size=128, shuffle=True
        )
        opt_C_pseudo = optim.Adam(global_C.parameters(), lr=0.001)
        for xb, yb in semi_loader:
            if xb.shape[0] < 2: continue
            xb, yb = xb.to(device), yb.to(device)
            preds = global_C(xb)
            loss = focal_loss(preds, yb)
            opt_C_pseudo.zero_grad(); loss.backward(); opt_C_pseudo.step()

# === Final Evaluation ===
def evaluate_model(model, X_test_tensor, y_test_tensor):
    model.eval()
    with torch.no_grad():
        preds = model(X_test_tensor.to(device))
        y_pred = preds.argmax(dim=1).cpu().numpy()
        y_true = y_test_tensor.cpu().numpy()

        acc = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
        recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
        f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

        print(f"\n✅ Accuracy: {acc}")
        print(f"🔍 Precision: {precision}")
        print(f"🔁 Recall: {recall}")
        print(f"🎯 F1-Score: {f1}")
        print("\n📊 Detailed Classification Report:\n", classification_report(y_true, y_pred))

evaluate_model(global_C, X_test_tensor, y_test_tensor)

# === Save Models ===
torch.save(global_G.state_dict(), "fgan2_generator_fm.pth")
torch.save(global_D.state_dict(), "fgan2_discriminator_fm.pth")
torch.save(global_C.state_dict(), "fgan2_classifier_fm.pth")



=== Epoch 1 ===


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Adding 3097 pseudo-labeled samples to training.

=== Epoch 2 ===
Adding 2080 pseudo-labeled samples to training.

=== Epoch 3 ===
Adding 5535 pseudo-labeled samples to training.

=== Epoch 4 ===
Adding 5993 pseudo-labeled samples to training.

=== Epoch 5 ===
Adding 5802 pseudo-labeled samples to training.

=== Epoch 6 ===
Adding 6551 pseudo-labeled samples to training.

=== Epoch 7 ===
Adding 5994 pseudo-labeled samples to training.

=== Epoch 8 ===
Adding 4465 pseudo-labeled samples to training.

=== Epoch 9 ===
Adding 5465 pseudo-labeled samples to training.

=== Epoch 10 ===
Adding 5317 pseudo-labeled samples to training.

=== Epoch 11 ===
Adding 5173 pseudo-labeled samples to training.

=== Epoch 12 ===
Adding 5065 pseudo-labeled samples to training.

=== Epoch 13 ===
Adding 4751 pseudo-labeled samples to training.

=== Epoch 14 ===
Adding 5484 pseudo-labeled samples to training.

=== Epoch 15 ===
Adding 5159 pseudo-labeled samples to training.

=== Epoch 16 ===
Adding 4701 pseudo